In [1]:
import pickle
from datetime import datetime
import json

In [2]:
folder = 'data/ple'

In [3]:
data_inicio = datetime.strptime('24/08/2020 00:00:00', '%d/%m/%Y %H:%M:%S')
turmas_validas = [12, 13, 14, 16, 17, 18, 20, 22, 23]

# Remove professores e usuarios que já tenham utilizado o sistema em outra turma
professores = Professor.objects.all().values_list('user')
usuarios_tentativa_anterior = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas,
                                                     timestamp__lt=data_inicio).values_list('user').distinct()

/home/lmoraes/.pyenv/versions/machineteaching/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1420: RuntimeWarning: DateTimeField UserLog.timestamp received a naive datetime (2020-08-24 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [4]:
# O que fazer com alunos repetentes?
# Vou separá-los por enquanto

In [16]:
%%time
userlog = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class')
print(userlog.count())

userlog_extra = UserLog.objects.filter(user__userprofile__id__in=u_final,
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class')

print(userlog_extra.count())

45295
1207
CPU times: user 13.4 ms, sys: 1.65 ms, total: 15.1 ms
Wall time: 534 ms


In [17]:
userlog = list(userlog) + list(userlog_extra)
len(userlog)

46502

In [18]:
with open("%s/userlogs.pkl" % folder, "wb") as pklfile:
    pickle.dump(list(userlog), pklfile)

In [20]:
%%time
userlog_complete = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class',
                                         'solution', 'seconds_in_page', 'seconds_in_code', 
                                         'seconds_to_begin')
print(userlog_complete.count())

userlog_complete_extra = UserLog.objects.filter(user__userprofile__id__in=u_final,
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class',
                                         'solution', 'seconds_in_page', 'seconds_in_code', 
                                         'seconds_to_begin')
print(userlog_complete_extra.count())

with open("%s/userlogs_complete.pkl" % folder, "wb") as pklfile:
    pickle.dump(list(userlog_complete) + list(userlog_complete_extra), pklfile)

45295
1207
CPU times: user 467 ms, sys: 42.7 ms, total: 510 ms
Wall time: 51.1 s


In [8]:
data_inicio = datetime.strptime('01/12/2020 00:00:00', '%d/%m/%Y %H:%M:%S')
userprofile_h = HistoricalUserProfile.objects.filter(history_date__gte=data_inicio, user__date_joined__lte=data_inicio)
u_to_consider = []
for idx, u in enumerate(userprofile_h):
    u_atual = UserProfile.objects.get(pk=u.id)
    if (u_atual.user_class != u.user_class) and u.user_class is not None and u.user_class.id in turmas_validas:
#         print(u_atual.user_class)
#         print(u.user_class)
        u_to_consider.append(u.id)
    if not (idx % 100):
        print(idx)

/home/lmoraes/.pyenv/versions/machineteaching/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1420: RuntimeWarning: DateTimeField HistoricalUserProfile.history_date received a naive datetime (2020-12-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/lmoraes/.pyenv/versions/machineteaching/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1420: RuntimeWarning: DateTimeField User.date_joined received a naive datetime (2020-12-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


0
EP1-EPT - João Vitor
Comp1 EB1 - PLE
EP1-EPT - João Vitor
Comp1 EB1 - PLE
EP1-EPT - João Vitor
Comp1 EB1 - PLE
EP1-EPT - João Vitor
Comp1 EB1 - PLE
EP1-EPT - João Vitor
Comp1 EB1 - PLE
EP1-EPT - João Vitor
Comp I EP - EB2 - 2893 - PLE120
100
MAB121_CMT_23765_REM2020_1
Comp I EP - EB2 - 2893 - PLE120
200
EE1 - Anamaria
Comp1 EB1 - PLE
EE1 - Anamaria
Comp1 EB1 - PLE


In [14]:
for u in u_to_consider:
    up = UserProfile.objects.get(pk=u)
    print(up.id, up.user)
    logs = UserLog.objects.filter(user=up.user, timestamp__lte=data_inicio).order_by('timestamp')
    print(logs[0].timestamp)
    print(logs[logs.count()-1].timestamp)
    print()

686 Henrique.ipa@poli.ufrj.br
2020-09-10 12:06:13.659747+00:00
2020-10-06 16:20:05.080377+00:00

686 Henrique.ipa@poli.ufrj.br
2020-09-10 12:06:13.659747+00:00
2020-10-06 16:20:05.080377+00:00

685 kaducosme@poli.ufrj.br
2020-09-10 12:08:08.178087+00:00
2020-10-04 01:46:26.470441+00:00

686 Henrique.ipa@poli.ufrj.br
2020-09-10 12:06:13.659747+00:00
2020-10-06 16:20:05.080377+00:00

686 Henrique.ipa@poli.ufrj.br
2020-09-10 12:06:13.659747+00:00
2020-10-06 16:20:05.080377+00:00

656 thaisbittencourt@poli.ufrj.br
2020-09-11 02:29:44.097080+00:00
2020-10-28 17:29:42.904666+00:00

606 bfcosta@dcc.ufrj.br
2020-08-04 18:40:46.293306+00:00
2020-11-27 19:10:47.389793+00:00

626 anamaria@dcc.ufrj.br
2020-08-04 18:48:22.429432+00:00
2020-10-07 20:10:24.053860+00:00

626 anamaria@dcc.ufrj.br
2020-08-04 18:48:22.429432+00:00
2020-10-07 20:10:24.053860+00:00



In [15]:
u_final = [686, 685, 656]

In [8]:
%%time
problems = set(list(userlog.values_list("problem__id", flat=True).order_by("id").distinct()))

CPU times: user 54.2 ms, sys: 8.79 ms, total: 63 ms
Wall time: 2.05 s


In [9]:
# One row per test case
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values(
    "problem_id", "content", "header", "problem__testcase__content").order_by("content")

In [10]:
%%time
data = []
old_content = ""
for item in solutions:
    # If it is the same problem, keep adding test cases
    if item["content"] == old_content:
        data[-1]["test_case"].append(json.loads(item["problem__testcase__content"]))
    # Otherwise, start new row
    else:
        item_data = {
            "problem_id": item["problem_id"],
            "solution": item["content"],
            "function_call": item["header"],
            "test_case": [json.loads(item["problem__testcase__content"])]
        }
        data.append(item_data)
    old_content = item["content"]

CPU times: user 13.4 ms, sys: 2.08 ms, total: 15.5 ms
Wall time: 1.14 s


In [11]:
%%time
# Transform it into dict to lookup for problem
data_dict = {}
for item in data:
    data_dict[item["problem_id"]] = item

CPU times: user 28 µs, sys: 19 µs, total: 47 µs
Wall time: 49.6 µs


In [ ]:
# Calculate test case percentage for each userlog
userlog_testcase = []
for item in userlog:
    tcs = data[userlog.problem_id]["test_case"]
    correct = 0
    # Calculate student response and solution for each test case for each log
    for tc in tcs:
        # Transform solution into Python function
        function_obj = compile(data[userlog.problem_id]["solution"], 'solution', 'exec')
        exec(function_obj)
        answer_solution = eval(item["function_call"])(*tc)
        
        # Transform student solution into Python function
        
        function_obj = compile(userlog.solution, 'solution', 'exec')
        exec(function_obj)
        answer_student = eval(item["function_call"])(*tc)

In [ ]:
for item in data:
    for tc in item["test_case"]:
        print(tc)
        # Transform solution into python function
        function_obj = compile(item["solution"], 'solution', 'exec')
        exec(function_obj)
        answer = eval(item["function_call"])(*tc)
        